In [1]:
!pip install twilio

StatementMeta(, 96047386-8225-45d1-b92a-120e3cdd76fe, 3, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.4 MB/s eta 0:00:00a 0:00:010m


In [2]:
from twilio.rest import Client
import http.client
import json
from datetime import datetime
import requests

StatementMeta(, 96047386-8225-45d1-b92a-120e3cdd76fe, 4, Finished, Available, Finished)

In [ ]:
def send_sos_message():
    account_sid = '<account_sid>'
    auth_token = '<auth_token>'
    client = Client(account_sid, auth_token)
    message = client.messages.create(
    messaging_service_sid='<messaging_service_sid>',
    body='There is an emergency.............. 👋',
    to='<phone_number>'
    )
    if message.sid :
        print("Message has been send")

StatementMeta(, 96047386-8225-45d1-b92a-120e3cdd76fe, 5, Finished, Available, Finished)

In [ ]:

def get_nearest_help_location(lat, long, type, radius): 
    url = "https://google-map-places.p.rapidapi.com/maps/api/place/nearbysearch/json"
    querystring = {"location":f"{lat},{long}","radius":f"{radius}","language":"en","opennow":"true","type" : type,"rankby":"prominence"}
    headers = {"x-rapidapi-key": "<token_key>","x-rapidapi-host": "google-map-places.p.rapidapi.com"}
    response = requests.get(url, headers=headers, params=querystring)
    places = response.json()

    results = []  # Initialize the results list

    for place in places.get('results', []):
        name = place.get('name')
        address = place.get('vicinity')
        if name and address:
            results.append({'name': name, 'address': address})

    try :
        if results[0] :
            return f"Name: {results[0]['name']}, Address: {results[0]['address']}"      
    except : 
        get_nearest_help_location(latitude, longitude, type, radius = 10000)

# get_nearest_help_location(18.4758, 73.84389, "police", radius=2000)


StatementMeta(, 96047386-8225-45d1-b92a-120e3cdd76fe, 6, Finished, Available, Finished)

In [ ]:

# Example of query for reading data from Kusto. Replace T with your <tablename>.
kustoQuery = f"panic_logs | where user_id == 1 | sort by session_id desc | top 1 by session_id"
    # The query URI for reading the data e.g. https://<>.kusto.data.microsoft.com.
kustoUri = "https://<cluster_id>.kusto.fabric.microsoft.com"
    # The database with data to be read.
database = "KQL_DB"
    # The access credentials.
accessToken = mssparkutils.credentials.getToken(kustoUri)
kustoDf  = spark.read\
        .format("com.microsoft.kusto.spark.synapse.datasource")\
        .option("accessToken", accessToken)\
        .option("kustoCluster", kustoUri)\
        .option("kustoDatabase", database)\
        .option("kustoQuery", kustoQuery).load()

# calling all the alerts 
# send_sos_message()

latitude = list(kustoDf.select("panic_latitude").collect())[0][0]
longitude = list(kustoDf.select("panic_longitude").collect())[0][0]

hospital = get_nearest_help_location(
    lat = latitude,
    long = longitude,
    type = "hospital",
    radius = 2000
)

police_station = get_nearest_help_location(
    lat = latitude,
    long = longitude,
    type = "police",
    radius = 2000
)

if hospital: 
    pass

if not police_station: 
    police_station = get_nearest_help_location(
        lat = latitude,
        long = longitude,
        type = "police",
        radius = 10000
    )

print(hospital, police_station)
currrent_timestamp = datetime.now()
df = spark.createDataFrame(
        [(1, 1, f"SOS Message send at {currrent_timestamp}, Nearest hosiptal {hospital} and police station is {police_station}", f"{currrent_timestamp}")], 
        ["alert_id", "user_id", "alert_message", "alert_send_at"]
    )
df.write.\
        format("com.microsoft.kusto.spark.synapse.datasource").\
        option("kustoCluster", kustoUri).\
        option("kustoDatabase", database).\
        option("kustoTable", "alerts_logs").\
        option("accessToken", accessToken).\
        mode("Append").save()
print("SENT")

StatementMeta(, 96047386-8225-45d1-b92a-120e3cdd76fe, 12, Finished, Available, Finished)

Name: SAMARTH MEDICO & SUPER SHOPEE, Address: venkateshwara tower,shop no 10,bharati vidyapeeth, Pune Name: Kondhwa Police Station, Address: FV8R+C6H, Katraj - Kondhwa Road, Kondhwa Budruk, Pune
SENT
